# Fastjet : Vectorizing Jet Finding

<br>
<p> The C++ fastjet library, along with the several Python interfaces. All of these interfaces have been used for a long time for Jet finding. However all the available options lack multi-event level vectorization among several other things. </p>
<UL>
    <li><p> Work on this started in February, 2021 </li>
    <li><p> Made to make use of the several new Scikit-HEP technologies. </p></li>
    <li><p>This marks the first time that the <b>official Fastjet bindings</b> will be available through pypi. Which means, you can : <br></p> <center> <h1> pip install fastjet </h1> </center> <br></li>
    <li> The package comes with all the bells and whistles of the C++ version of Fastjet. (All the plugins and CGAL support enabled) </li>
    <li> We intend to add FJ Contrib in future as well. </li>

<br><br>
# Pyjet

<p> Pyjet is a popular Python package for using FastJet with NumPy.</p>
<p>Uses Numpy Arrays to provide event level vectorization. </p>

However, it has two problems:
    <UL>
        <li> It takes <b>multiple function calls</b> to process multiple events. </li>
        <li> It uses <b>FJ Core</b>, the header version of Fastjet.</li>

In [1]:
from pyjet import cluster
from pyjet.testdata import get_event

vectors = get_event()
sequence = cluster(vectors, R=1.0, p=-1)
jets = sequence.inclusive_jets()  # list of PseudoJets
exclusivejets = sequence.exclusive_jets(3)  
exclusivejets

[PseudoJet(pt=0.050, eta=-8.038, phi=-1.719, mass=0.019),
 PseudoJet(pt=0.013, eta=-6.696, phi=-1.447, mass=0.000),
 PseudoJet(pt=0.011, eta=-9.489, phi=-1.408, mass=0.000)]

<br><br>
# The New fastjet Interface

<p> The new Fastjet interface has been fashioned from the ground up to address these critical issues. </p>

<p><B>How?</B></p>
<UL>
    <li> We have compiled and put the <B>full Fastjet C++ library </B> on the Python Wheel </li>
    <li> We have used <B> Awkward Arrays </B> as the mode of input and output to handle Multi-event (Jagged) cases. </li>
</UL>

![talk_graphic](talk_graphic4.png)

The new interface for Fastjet contains two kinds of interfaces:
 * <b>Classic interface </b> (Taken from the python interface generated by the C++ library.)
 * <b>Vectorized interface</b> (Awkward Interface)

<br><br>
# The Classic Interface

<p> The classic interface is the Swig interface that can be downloaded and compiled from the C++ version of fastjet.</p>
<p>Since we were compiling against the whole library, it was easy enough to add the Swig interface to the inventory of the new package. </p>
<p> There is an almost complete overlap in the methods for the C++ and the Python classic interface.</p>

In [2]:
import fastjet
from fastjet import ClusterSequence

In [3]:
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.6)
particles = [fastjet.PseudoJet(0.7283822313,0.1094889185 ,-0.1754670827  ,-0.6843130641)
            ,fastjet.PseudoJet(6.7420362761,-1.3395280577,-1.0677537402,-6.4527380155)
            ,fastjet.PseudoJet(0.9647588756,-0.4610425267 ,-0.0168193957 ,-0.8357253135)
            ,fastjet.PseudoJet(6.5019392609 ,-0.8586330562 ,-2.0252604440 ,-6.1169276419)
            ,fastjet.PseudoJet(0.4622264166,0.1506846470 ,-0.2812835366 ,-0.3038867955)]
cluster = fastjet.ClusterSequence(particles,jetdef)

In [4]:
inc_jets = cluster.inclusive_jets()
for i in range(len(inc_jets)):
    print('px: {px}, py: {py}, pz: {pz}, E: {E} '.format(px=inc_jets[i].px(),
            py=inc_jets[i].py(),pz=inc_jets[i].pz(),E=inc_jets[i].E()))

px: 0.4622264166, py: 0.150684647, pz: -0.2812835366, E: -0.3038867955 
px: 14.9371166439, py: -2.5497147221, pz: -3.2853006626, E: -14.089704034999999 


The major characteristics of the classsical interface:
<Ul>
    <li><p> It does things one event at a time.</p></li>
    <li><p> Things are done almost similarly to how they are done in C++, by constructing lists of PseudoJets and passing them one by one. </p></li>


<br><br>
# The Vectorized Interface

<p> The vectorized interface uses Awkward Arrays as the sole input data type for particle data.</p>
<p> The outputs are also Awkward Arrays wherever possible.</p> 
<p> The use of Awkward Arrays has allowed us to introduce Multi-Event Vectorized Jet Finding. </p>
<p> The Python Classic interface and the Awkward interface also have a big overlap. </p>
<p> Which means that we can use the already available classes in the classical interface to provide clustering specifications like JetDefinition and AreaDefintion. </p>



In [5]:
import awkward as ak
array = ak.Array(
    [
        [
            {"px": 0.45417, "py": 0.32728, "pz": -0.23645, "E": 0.15539, "charge": 1},
            {"px": 1.89939, "py": -0.78538, "pz": -0.78105, "E": -1.53662, "charge": 1},
            {"px": 3.3957985464, "py": 3.20144, "pz": -0.31280, "E": -0.55188, "charge": 1},
        ],
        [],
        [
            {"px": 2.95593, "py": -0.35361, "pz": 0.62956, "E": -2.86073, "charge": 1},
            {"px": 4.33707, "py": 0.53039, "pz": 1.47772, "E": -3.93265, "charge": 1},
            {"px": 0.32284, "py": 0.06157, "pz": 0.12261, "E": -0.25641, "charge": 1},
            {"px": 0.32114, "py": 0.01579, "pz": 0.01635, "E": -0.32380, "charge": 1},
        ],
        [
            {"px": 9.7436, "py": -0.0011, "pz": 0.238, "E": -9.7392, "charge": 1}
        ],
    ],
)

<p> <B>Note : </B> the JetDefinition and the Clustering class is the same as was used for the Classic interface </p>

In [6]:
cluster = fastjet.ClusterSequence(array, jetdef)
cluster.inclusive_jets().to_list()

[[{'px': 0.45417, 'py': 0.32728, 'pz': -0.23645, 'E': 0.15539},
  {'px': 1.89939, 'py': -0.78538, 'pz': -0.78105, 'E': -1.53662},
  {'px': 3.3957985464, 'py': 3.20144, 'pz': -0.3128, 'E': -0.55188}],
 [],
 [{'px': 7.93698,
   'py': 0.25414000000000003,
   'pz': 2.2462400000000002,
   'E': -7.373590000000001}],
 [{'px': 9.7436, 'py': -0.0011, 'pz': 0.238, 'E': -9.7392}]]

In [7]:
cluster.inclusive_jets().type

4 * var * {"px": float64, "py": float64, "pz": float64, "E": float64}

<p> The data is exactly the same, the structure has been redefined by Jet Clustering. </p>

In [8]:
output = cluster.constituents()
output.to_list()

[[[{'px': 0.45417, 'py': 0.32728, 'pz': -0.23645, 'E': 0.15539, 'charge': 1}],
  [{'px': 1.89939,
    'py': -0.78538,
    'pz': -0.78105,
    'E': -1.53662,
    'charge': 1}],
  [{'px': 3.3957985464,
    'py': 3.20144,
    'pz': -0.3128,
    'E': -0.55188,
    'charge': 1}]],
 [],
 [[{'px': 2.95593, 'py': -0.35361, 'pz': 0.62956, 'E': -2.86073, 'charge': 1},
   {'px': 4.33707, 'py': 0.53039, 'pz': 1.47772, 'E': -3.93265, 'charge': 1},
   {'px': 0.32284, 'py': 0.06157, 'pz': 0.12261, 'E': -0.25641, 'charge': 1},
   {'px': 0.32114, 'py': 0.01579, 'pz': 0.01635, 'E': -0.3238, 'charge': 1}]],
 [[{'px': 9.7436, 'py': -0.0011, 'pz': 0.238, 'E': -9.7392, 'charge': 1}]]]

In [9]:
output.type

4 * var * var * {"px": float64, "py": float64, "pz": float64, "E": float64, "charge": int64}

<p> The availability of Awkward Array allows the user to manipulate the data with relative ease. </p>

<p>The input data had an extra parameter from the start, the ClusterSequence class ignored it while preserving the data association. </p>
<p> Now, the extra parameter(s) can be used to calculate the charge (in this particular case) of a Jet itself ! </p>

In [10]:
ak.sum(output.charge, axis=-1)

<Array [[1, 1, 1], [], [4], [1]] type='4 * var * int64'>

<br><br>
# Vector 

<p> All the functions of the PseudoJet class can be performed using the Vector Python library. </p>
<p> Vector is a Python library for 2D , 3D and Lorentz Vectors. </p>

<p> A valid Awkward Array input needs to satisfy either of these two conditions :
    <UL>
        <li> Have the coordinate system as px, py, pz and E. </li>
        <li> Have a Vector Backend and name as "Momentum4D". </li>
    </UL>
</p>

In [11]:
import vector
vector.register_awkward()

In [12]:

input_data = ak.Array(
    [[
            {"pt": 1.2, "eta": 3.2, "phi": 2.14, "mass": 0.25},
            {"pt": 32.2, "eta": -3.1, "phi": 1.67, "mass": 0.2412},
            {"pt": 32.45, "eta": -3.14, "phi": 1.66, "mass": 0.2456},
    ],
    [
            {"pt": 1.2, "eta": -4.2, "phi": -2.89, "mass": 0.25},
            {"pt": 32.2, "eta": -4.21, "phi": -2.891, "mass": 0.2412},
            {"pt": 32.45, "eta": 1.25, "phi": 1.68, "mass": 0.2456},
    ]],
    with_name = "Momentum4D", # This line specifies that this is a lorentz vector
)

<p> Vector allows any one to convert the coordinate system of any data easily, provided the right name has been given to the array </p>
<p> The lorentz vectors can be extracted from the array by simply using the name of the coordinate as an attribute : </p>

In [13]:
input_data.px, input_data.py, input_data.pz, input_data.E 

(<Array [[-0.647, -3.19, ... -31.2, -3.54]] type='2 * var * float64'>,
 <Array [[1.01, 32, 32.3, ... -7.98, 32.3]] type='2 * var * float64'>,
 <Array [[14.7, -357, -374, ... -1.08e+03, 52]] type='2 * var * float64'>,
 <Array [[14.7, 358, 376, ... 1.08e+03, 61.3]] type='2 * var * float64'>)

<p> Awkward Arrays with the vector backend do not need to be in any specific coordinate system, as long as they are named "Momentum4D" </p>

In [18]:
cluster = fastjet.ClusterSequence(input_data, jetdef)
constituents = cluster.constituents()
cluster.inclusive_jets().to_list()

[[{'px': -0.6467537392794138,
   'py': 1.0107965179639748,
   'pz': 14.69506079587859,
   'E': 14.746094798100543},
  {'px': -6.079943158067136,
   'py': 64.36266233667942,
   'pz': -730.8196998594822,
   'E': 733.6749725017362}],
 [{'px': -3.536620110183818,
   'py': 32.25670191132757,
   'pz': 51.98227415576179,
   'E': 61.27984697896023},
  {'px': -32.35647377729993,
   'py': -8.283632419349921,
   'pz': -1124.2040656910965,
   'E': 1124.7009210979961}]]

In [15]:
constituents.to_list()

[[[{'pt': 1.2, 'eta': 3.2, 'phi': 2.14, 'mass': 0.25}],
  [{'pt': 32.2, 'eta': -3.1, 'phi': 1.67, 'mass': 0.2412},
   {'pt': 32.45, 'eta': -3.14, 'phi': 1.66, 'mass': 0.2456}]],
 [[{'pt': 32.45, 'eta': 1.25, 'phi': 1.68, 'mass': 0.2456}],
  [{'pt': 1.2, 'eta': -4.2, 'phi': -2.89, 'mass': 0.25},
   {'pt': 32.2, 'eta': -4.21, 'phi': -2.891, 'mass': 0.2412}]]]

<br><br>
# Conclusion

* You can <b> pip install fastjet </b> soon.
* There is multilevel vectorized clustering.
* Handling the data was never this easy!